# IESB - Trabalho final - Data Mining e Machine Learning II
## Marcilon Silva Cunha Alves 
## Matricula: 1931133129

# Previsão da situação de pobreza Costa-Riquenha

O Banco Interamericano de Desenvolvimento está pedindo à comunidade Kaggle ajuda com a qualificação de renda para algumas das famílias mais pobres do mundo. 

Na América Latina, um método popular usa um algoritmo para verificar a qualificação de renda. É chamado de Teste de Média de Proxy (ou PMT). Com a PMT, as agências usam um modelo que considera os atributos domésticos observáveis de uma família, como o material das paredes e do teto, ou os ativos encontrados na casa para classificá-los e prever seu nível de necessidade.

Para melhorar a PMT, o BID (a maior fonte de financiamento para o desenvolvimento da América Latina e do Caribe) recorreu à comunidade Kaggle. Eles acreditam que novos métodos além da econometria tradicional, com base em um conjunto de dados de características domésticas da Costa Rica, podem ajudar a melhorar o desempenho do PMT.

Além da Costa Rica, muitos países enfrentam o mesmo problema de avaliar incorretamente as necessidades sociais. Se Kagglers puder gerar uma melhoria, o novo algoritmo pode ser implementado em outros países ao redor do mundo.


## Carregar e ler dados

* Importanto das bibliotecas necessárias
* Listando os arquivos de banco de dados disponível no diretório input

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

* Carregando os dados de treino e de teste

In [ ]:
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

* Juntando os dataframes df (treino) e test (teste)

In [ ]:
df_all = df.append(test)

df_all.shape

## Análise exploratória e tratamento de dados

* Estatísticas básicas
* Verificar mínimos e máximos para garantir se estão dentro dos limites esperados
* Verificar intervalo de variação da medida
* Verificar possíveis outliers

Analisando a variável TARGET com a base de dados Train.

Os valores Target representam os níveis de pobreza da seguinte forma:

1 = pobreza extrema;

2 = pobreza moderada;

3 = famílias vulneráveis;

4 = famílias não vulneráveis;


In [ ]:
df['Target'].hist(grid = False, bins = 10)

In [ ]:
df.Target.value_counts()/100

* Verificar os principais indicadores descritivos

In [ ]:
df_all.describe().T

* Verificando quais colunas do dataframe são do tipo object

In [ ]:
df_all.select_dtypes('object').head()

* Observando os dados da coluna edjefa


In [ ]:
df_all['edjefa'].value_counts()

* Transformando 'yes' em 1 e 'no' em 0
* Para as colunas edjefa e edjefe

In [ ]:
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

* Observando as colunas do dataframe que são do tipo object

In [ ]:
df_all.select_dtypes('object').head()

* Obserando a coluna dependency

In [ ]:
df_all['dependency'].value_counts()

* Transformando 'yes' em 1 e 'no' em 0
* Para a coluna dependency

In [ ]:
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

* Observando quais colunas do dataframe são do tipo object

In [ ]:
df_all.select_dtypes('object').head()

* Analisando os valores nulos de cada variável

In [ ]:
df_all.isnull().sum().sort_values()

In [ ]:
data_na = df_all.isnull().sum().values / df_all.shape[0] *100
df_na = pd.DataFrame(data_na, index=df_all.columns, columns=['Count'])
df_na = df_na.sort_values(by=['Count'], ascending=False)

missing_value_count = df_na[df_na['Count']>0].shape[0]

print(f'We got {missing_value_count} rows which have missing value in train set ')
df_na.head(6)

# rez_esc represents "years behind in school", missing value could be filled as 0
# meaneduc represents "average years of education for adults (18+)", missing value could be filled as 0
# v18q1 really depends on v18q
# v2a1 depends on tipovivi3
# We do not really need SQBxxxx features for polynomial in our case, and i will use fillna as 0 after at the last step of feature engineering


* Analisando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)

In [ ]:
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

* Analizando os dados de variável v18q

In [ ]:
df_all['v18q'].value_counts()

* Prenchendo com "0" os valores nulos de: v2a1, v18q1 e rez_esc

In [ ]:
df_all['v2a1'].fillna(0, inplace=True)
df_all['v18q1'].fillna(0, inplace=True)
df_all['rez_esc'].fillna(0, inplace=True)


* Plotanto v2a1 e analisando distribuição

In [ ]:
df_all['v2a1'].hist(grid = False, bins = 10)

* Plotanto v18q1 e analisando distribuição

In [ ]:
df_all['v18q1'].hist(grid = False, bins = 10)

* Analizando os principais indicadores para meaneduc

In [ ]:
df_all.meaneduc.describe().T

* Analizando os principais indicadores para SQBmeaned

In [ ]:
df_all.SQBmeaned.describe().T

* Prenchendo com a Mediana os valores nulos de: meaneduc e SQBmeaned

In [ ]:
#df_all.loc[df_all.meaneduc.isnull(), "meaneduc"] = 0
#df_all.loc[df_all.SQBmeaned.isnull(), "SQBmeaned"] = 0

df_all['meaneduc'].fillna(df_all['meaneduc'].median(), inplace=True) 
df_all['SQBmeaned'].fillna(df_all['SQBmeaned'].median(), inplace=True)


* Plotanto meaneduc e analisando distribuição

In [ ]:
df_all['meaneduc'].hist(grid = False, bins = 10)

* Plotanto SQBmeaned e analisando distribuição

In [ ]:
df_all['SQBmeaned'].hist(grid = False, bins = 10)

* Verificando quais variáveis ainda possuem nulos

In [ ]:
df_all.isnull().sum().sort_values()

* Prenchendo com "-1" os valores nulos ainda existentes

In [ ]:
df_all.fillna(-1, inplace=True)


* Feature Engineering

* Criando novas colunas para valores percapita

In [ ]:
df_all['hsize-pc'] = df_all['hhsize'] / df_all['tamviv']
df_all['phone-pc'] = df_all['qmobilephone'] / df_all['tamviv']
df_all['tablets-pc'] = df_all['v18q1'] / df_all['tamviv']
df_all['rooms-pc'] = df_all['rooms'] / df_all['tamviv']
df_all['rent-pc'] = df_all['v2a1'] / df_all['tamviv']

* Plotando a Matrix de correlação

In [ ]:

import seaborn as sns

variables = ['Target', 'dependency', 'v2a1', 'v18q1', 'rez_esc', 'meaneduc' ,'SQBmeaned']

# Calculate the correlations
corr_mat = df_all[variables].corr().round(2)

# Draw a correlation heatmap
plt.rcParams['font.size'] = 12
plt.figure(figsize = (12, 12))
sns.heatmap(corr_mat, vmin = -0.5, vmax = 0.8, center = 0, 
            cmap = plt.cm.RdYlBu, annot = True);



* Separando as colunas para treinamento

In [ ]:

feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

* Separando os dataframes

In [ ]:
train, test = df_all[df_all['Target'] != -1], df_all[df_all['Target'] == -1]

Embora todos os membros de uma família devam ter o mesmo rótulo nos dados de treinamento, existem erros onde os indivíduos na mesma casa têm rótulos diferentes. Nestes casos, somos orientados a usar o rótulo do chefe de cada família, que pode ser identificado pelas linhas onde parentesco1 == 1.0. 

* Limitando o treinamento ao chefe da familia
* Criando um novo dataframe para treinar

In [ ]:
heads = train[train['parentesco1'] == 1]

# MODELOS


## LightGBM

* Instanciando o LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
#parameter value is copied from 
clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.1, objective='multiclass',
                             random_state=None, silent=True, metric='None', 
                             n_jobs=4, n_estimators=700, class_weight='balanced',
                             colsample_bytree =  0.93, min_child_samples = 95, num_leaves = 14, subsample = 0.96)


clf.fit(heads[feats], heads['Target'])

accuracy_score(heads['Target'], clf.predict(heads[feats]))

In [ ]:
test['Target'] = clf.predict(test[feats]).astype(int)


In [ ]:
test['Target'].value_counts(normalize=True)

In [ ]:

#test[['Id', 'Target']].to_csv('submission.csv', index=False)

## CatBoostClassifier

* Instanciando o CBC

In [ ]:
# Trabalhando com CatBoost
from catboost import CatBoostClassifier
cbc = CatBoostClassifier(random_state=42)
cbc.fit(heads[feats], heads['Target'])
accuracy_score(test['Target'], cbc.predict(test[feats]))

In [ ]:
test['Target'] = cbc.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)
#test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
fig=plt.figure(figsize=(15, 20))

pd.Series(cbc.feature_importances_, index=feats).sort_values().plot.barh()

## RandomForestClassifier

* Instanciando o random forest classifier

In [ ]:


rf = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')


* Treinando o modelo RandomForestClassifier

In [ ]:
rf.fit(heads[feats], heads['Target'])

* Prevendo o Target de teste usando o modelo treinado

In [ ]:
test['Target'] = rf.predict(test[feats]).astype(int)

* Analisando as previsões para o Target

In [ ]:
test['Target'].value_counts(normalize=True)


In [ ]:
accuracy_score(heads['Target'], rf.predict(heads[feats]))

* Criando o arquivo para submissão

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)


* Plotando e avalisando a importância de cada coluna (cada variável de entrada)

In [ ]:
fig=plt.figure(figsize=(15, 20))

pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 18)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 12)
    plt.yticks(tick_marks, classes, size = 12)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize = 16,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 12)
    plt.xlabel('Predicted label', size = 12)


In [ ]:
    cm = confusion_matrix(heads['Target'], rf.predict(heads[feats]))

    plot_confusion_matrix(cm, classes = ['Extreme', 'Moderate', 'Vulnerable', 'Non-Vulnerable'],
                      title = 'Poverty Confusion Matrix')

## Conclusão

Esse desafio foi de grande proveito para a prática dos conhecimentos adquiridos, pois foi possível aplicar uma solução completa de ciência de dados para um problema do mundo real. 

Caminho percorrido para chegar nessa solução:
* Entendimento do desafio;
* Análise Exploratória de Dados;
* Análise dos dados com problemas e pesquisa de soluções propostas nos fóruns de discussões;
* Preenchimento dos valores ausentes utilizando técnicas de input;
* Criação de novas feactures;
* Experimento de modelos diferentes para verificar qual apresentava melhor score;

Após aplicação dos modelos foi possível analisar os resultados, principalmente quais variáveis mais influenciavam nos resultados. 

Com isso foi possível efetuar novas tentativas com inputs diferenciados para essas variáveis em questão, ação essa que melhorou significativamente os resultados obtidos.

### RESULTADOS

Modelo LightGBM
* Score: 0.37500

Modelo CatBoostClassifier
* Score: 0.37500

Modelo RandomForestClassifier
* Score: 0.44117

A parte mais valiosa do trabalho proposto foi o conhecimento prático adquirido com a utilização e participação efetiva nas competições do Kaggle. Foi ótimo desenvolver soluções realistas como um cientista de dados. 


Marcilon Cunha.
